Vimos como adaptar um conjunto de dados por alguns modelos específicos.
De uma maneira geral, podemos considerar um modelo da forma
$$ f(x) = \alpha_1 \phi_1(x) + \dots + \alpha_n \phi_n(x), $$
e fazer o mesmo procedimento.

Esse problema com esse modelo é dito **quadrados mínimos lineares discretos.**

Para encontrar os parâmetros $\alpha$ fazemos o mesmo procedimento anterior:
$$ \min E(\alpha) = \frac{1}{2} \sum_{i=1}^m [f(x_i) - y_i]^2
= \frac{1}{2} \sum_{i=1}^m \bigg[\sum_{j=1}^n \alpha_j\phi_j(x_i) - y_i\bigg]^2. $$

Derivando em cada $\alpha_k$ nos dá um sistema $M\alpha = c$, onde
$$ M_{kj} = \sum_{i=1}^m \phi_k(x_i)\phi_j(x_i) \qquad \mbox{e}
\qquad c_j = \sum_{i=1}^m \phi_j(x_i) y_i. $$

Dados a função $\phi_j$ e os pontos $x_i$, podemos definir um vetor $\overline{\phi}_j = (\phi_j(x_1), \dots, \phi_j(x_m))^T$, isto é, o vetor obtido aplicando $\phi_j$ em cada elemento $x_i$.
Com essa definição, podemos escrever
$$ M_{k,j} = \langle\overline{\phi}_k,\overline{\phi}_j\rangle \qquad \mbox{e} \qquad
c_j = \langle\overline{\phi}_j, y\rangle. $$

O sistema $M\alpha = c$ é chamado de **sistema normal**, e ele tem uma característica muito importante:
A matriz $M$ é semidefinida positiva, e em muitos casos, definida positiva.

**Def.: ** Uma matriz simétrica $A$ é dita definida positiva se $v^TAv > 0$ para todo $v\neq 0$.

**Exercício: ** Demonstre que uma matriz simétrica definida positiva é não singular usando apenas a definição acima.

**Def.: ** Uma matriz simétrica $A$ é dita semi-definida positiva se $v^TAv \geq 0$ para todo $v$.

**Teo.: ** A é definida positiva se, e somente se, seus autovalores são positivos.

**Teo.: ** A é semi-definida positiva se, e somente se, seus autovalores são positivos ou nulos.

**Teo.: ** A é definida positiva se todos os determinantes $det(A[1:k,1:k])$ são positivos, para $k = 1,\dots,n$.

Existem muitos outros teoremas sobre definida positiva, mas um ponto importante aqui é que se $A$ é definida positiva, então ela é não-singular. Além disso, pelo teorema dos autovalores, sabemos como determinar se $A$ é definida positiva na prática.
Na prática, no entanto, as contas não valem a pena, e o teorema abaixo é muito mais útil:

**Teo.: ** $A$ é simétrica definida positiva se, e somente se, existe uma matriz $G$ quadrada não-singular tal que $A = GG^T$.

Podem existir mais de uma matriz $G$ que satisfaça essa propriedade, mas estamos preocupados com uma em particular:
A matriz da **decomposição de Cholesky**.

A decomposição de Cholesky de uma matriz $A$ simétrica definida positiva é a matriz $G$ triangular inferior que satisfaz $A = GG^T$
e com diagonal positiva.

In [1]:
G = [2 0; -1/2 sqrt(15)/2]

2x2 Array{Float64,2}:
  2.0  0.0    
 -0.5  1.93649

In [2]:
G*G'

2x2 Array{Float64,2}:
  4.0  -1.0
 -1.0   4.0

*Desenvolvimento feito em sala*

In [3]:
# Implementação
function cholesky(A)
    return chol(A)'
end

cholesky (generic function with 1 method)

In [4]:
A = [4.0  -1.0; -1.0  4.0]
cholesky(A)

2x2 LowerTriangular{Float64,Array{Float64,2}}:
  2.0  0.0    
 -0.5  1.93649

In [5]:
A = rand(10,10)
A = A*A' + eye(10)
G = cholesky(A)
norm(A - G*G', Inf)

1.3322676295501878e-15

## Forma matricial

Com os vetores $\overline{\phi}_1, \dots, \overline{\phi}$, podemos montar a matriz
$$ A = [\overline{\phi}_1 \cdots \overline{\phi}_n]. $$
Com essa matriz, podemos ver que $ A^TA = M$ e que $A^Ty = c$.
Ou seja, o sistema que queremos resolver é, na verdade
$$ A^TA\alpha = A^Tb, $$
ou seja, $A^T(A\alpha - b) = 0$.

Isso quer dizer que o sistema normal tem alguma relação com o sistema sobre-determinado $A\alpha = b$.
De fato, se pensarmos novamente no problema de minimizar aquela $E(\alpha)$, cada termo dentro do somatório é
uma linha de $A\alpha - b$.
Em outras palavras,
$$ E(\alpha) = \frac{1}{2}\sum_{i=1}^m (A\alpha-b)_i^2
= \frac{1}{2}\Vert A\alpha - b\Vert^2. $$

De fato, se calculamos $\nabla E(\alpha) = 0$, obtemos as equações normais.
Isso generaliza o problema de quadrados mínimos.
Esse problema aparece com frequência em outros contextos, e é bastante importante sabermos resolvê-lo eficientemente.

** Exercício: ** Calcule as derivadas parciais de $f(x) = \frac{1}{2}x^TQx + x^Tg$.
Escreva a forma do gradiente.

Computacionalmente, o problema de quadrados mínimos, como descrito acima, envolve o cálculo de $A^TA$, depois de Cholesky, e depois da resolução do sistema. Alguns códigos, em particular o [CHOLMOD](http://faculty.cse.tamu.edu/davis/suitesparse.html), calculam a decomposição de Cholesky diretamente da matriz $A$, o que reduz bastante o trabalho computacional.

Uma outra possibilidade de resolver esse problema seria usar algum método iterativo. Os métodos que vimos não são bons para isso, pois envolvem a formação da matriz $A$. No entanto, existe um outro método, chamado **Método dos Gradientes Conjugados**, que é bastante útil para esse problema.

In [6]:
A = rand(5000, 5);

In [7]:
v = rand(5)

5-element Array{Float64,1}:
 0.10974 
 0.884721
 0.751625
 0.28462 
 0.910323

In [8]:
# Pra fazer A'*A*v tem duas opções:

In [9]:
@time (A'*A) * v

  0.053890 seconds (85.25 k allocations: 3.868 MB, 6.97% gc time)


5-element Array{Float64,1}:
 3739.85
 4136.43
 3986.62
 3831.7 
 4113.02

In [10]:
@time A' * (A * v)

  0.002817 seconds (2.54 k allocations: 163.112 KB)


5-element Array{Float64,1}:
 3739.85
 4136.43
 3986.62
 3831.7 
 4113.02

In [11]:
function quadmin(F, x, y)
    n = length(F)
    m = length(x)
    A = zeros(m, n)
    for j = 1:n
        f = F[j]
        for i = 1:m
            A[i,j] = f(x[i])
        end
    end
    return (A'*A)\(A'*y)
end

quadmin (generic function with 1 method)

In [12]:
using Plots
gr()

m = 500
x = sort(5 * rand(m) - 2) # x ∈ [-2,3]
y = 2 + 0.2 * exp(x) - 0.5 * x + 0.3 * x.^2 + rand(m)*0.2

scatter(x, y, ms=3)

[Plots.jl] Initializing backend: gr


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [13]:
# O modelo é dado pelas funções
# f1(x) = 1, f2(x) = exp(x), f3(x) = x, f4(x) = x^2
F = [x->1, x->x, x->x^2, x->exp(x)]
α = quadmin(F, x, y)

scatter(x, y, ms=3)
plot!(t->dot(α, [f(t) for f in F]), x[1], x[end], c=:red, lw=2)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 4 
 
 
 6 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 51.3,220.142 56.6423,225.752 61.9847,231.256 67.3271,236.654 72.6695,241.944 78.0119,247.127 83.3543,252.203 88.6967,257.171 94.039,262.03 99.3814,266.781 
 104.724,271.422 110.066,275.955 115.409,280.377 120.751,284.69 126.093,288.891 131.436,292.982 136.778,296.961 142.121,300.828 147.463,304.582 152.805,308.222 
 158.148,311.749 163.49,315.161 168.832,318.458 174.175,321.639 179.517,324.704 184.86,327.65 190.202,330.479 195.544,333.188 200.887,335.777 206.229,338.245 
 211.572,340.591 216.914,342.814 222.256,344.912 227.599,346.885 232.941,348.731 238.283,350.449 243.626,352.038 248.968,353.496 254.311,354.822 259.653,356.013 
 264.995,357.069 270.338,357.988 275.68,358.767 281.023,359.405 286.365,359.9 291.707,360.249 297.05,360.451 302.392,360.503 307.734,360.402 313.077,360.145 
 318.419,359.731 323.762,359.156 329.104,358.417 334.446,357.511 339.789,356.434 345.131,355.182 350.474,353.753 355.816,352.142 361.158,350.345 366.501,348.358 
 371.843,346.175 377.185,343.793 382.528,341.206 387.87,338.408 393.213,335.395 398.555,332.16 403.897,328.697 409.24,325 414.582,321.061 419.925,316.873 
 425.267,312.43 430.609,307.722 435.952,302.742 441.294,297.48 446.636,291.928 451.979,286.075 457.321,279.912 462.664,273.427 468.006,266.609 473.348,259.447 
 478.691,251.927 484.033,244.037 489.376,235.762 494.718,227.089 500.06,218.001 505.403,208.482 510.745,198.517 516.087,188.086 521.43,177.17 526.772,165.752 
 532.115,153.808 537.457,141.318 542.799,128.259 548.142,114.607 553.484,100.335 558.827,85.4182 564.169,69.8277 569.511,53.5342 574.854,36.5064 580.196,18.712 
 
 "/>
 
 
 
 
 
 
 
 y1 
 
 
 
 y2

In [14]:
[f(2) for f in F]

4-element Array{Any,1}:
 1      
 2      
 4      
 7.38906

In [15]:
for f in F
    fx = f(1.2)
    println("f(1.2) = $fx")
end

f(1.2) = 1
f(1.2) = 1.2
f(1.2) = 1.44
f(1.2) = 3.3201169227365472


## Exercício para 10 de Outubro

Muitas vezes, além dos erros obtidos numa medição, existem os
**outliers**, isto é, dados que foram obtidos de *maneira errada*.
Talvez por algum instrumento defeituoso, ou alguma metodologia
que foi entendida errada.

In [16]:
m = 500
x = sort(5 * rand(m) - 2) # x ∈ [-2,3]
y = 2 - 0.5 * x + 0.3 * x.^2 + rand(m)*0.1 + [rand() < 0.05 ? 2 : 0 for i=1:m]

scatter(x, y, ms=3)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [17]:
A = [ones(x) x x.^2];
α = A\y
scatter(x, y, ms=3)
plot!(x->dot(α,[1,x,x^2]), x[1], x[end], c=:red, lw=2)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 51.3,124.608 56.6423,132.749 61.9847,140.742 67.3271,148.586 72.6695,156.282 78.0119,163.83 83.3543,171.23 88.6967,178.481 94.039,185.585 99.3814,192.54 
 104.724,199.347 110.066,206.006 115.409,212.516 120.751,218.879 126.093,225.093 131.436,231.159 136.778,237.077 142.121,242.847 147.463,248.468 152.805,253.942 
 158.148,259.267 163.49,264.444 168.832,269.473 174.175,274.353 179.517,279.086 184.86,283.67 190.202,288.106 195.544,292.394 200.887,296.534 206.229,300.525 
 211.572,304.368 216.914,308.064 222.256,311.61 227.599,315.009 232.941,318.26 238.283,321.362 243.626,324.316 248.968,327.122 254.311,329.78 259.653,332.29 
 264.995,334.651 270.338,336.865 275.68,338.93 281.023,340.847 286.365,342.616 291.707,344.236 297.05,345.708 302.392,347.033 307.734,348.209 313.077,349.237 
 318.419,350.116 323.762,350.848 329.104,351.431 334.446,351.866 339.789,352.153 345.131,352.292 350.474,352.282 355.816,352.124 361.158,351.819 366.501,351.365 
 371.843,350.762 377.185,350.012 382.528,349.113 387.87,348.067 393.213,346.872 398.555,345.529 403.897,344.037 409.24,342.398 414.582,340.61 419.925,338.674 
 425.267,336.59 430.609,334.358 435.952,331.978 441.294,329.449 446.636,326.772 451.979,323.947 457.321,320.974 462.664,317.853 468.006,314.583 473.348,311.165 
 478.691,307.6 484.033,303.885 489.376,300.023 494.718,296.013 500.06,291.854 505.403,287.547 510.745,283.092 516.087,278.489 521.43,273.738 526.772,268.838 
 532.115,263.79 537.457,258.595 542.799,253.25 548.142,247.758 553.484,242.118 558.827,236.329 564.169,230.392 569.511,224.307 574.854,218.074 580.196,211.693 
 
 "/>
 
 
 
 
 
 
 
 y1 
 
 
 
 y2

Estude alguma maneira de remover outliers de um conjunto de dados,
faça uma explicação dessa maneira, e faça um código que faz
essa remoção de outliers de um conjunto de dados. Depois, teste
com o problema dado.

Os passos são:
- resolva com outliers e plote
- remova outliers com o seu código
- resolva sem outliers e plote

In [20]:
r = A*α - y
scatter(x, r, ms=3)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 -1.5 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [27]:
mean(r), std(r)

(-1.4210854715202004e-17,0.510579862013265)

search: std stdm STDIN STDOUT STDERR setdiff setdiff! hist2d hist2d! stride



```
std(v[, region])
```

Compute the sample standard deviation of a vector or array `v`, optionally along dimensions in `region`. The algorithm returns an estimator of the generative distribution's standard deviation under the assumption that each entry of `v` is an IID drawn from that generative distribution. This computation is equivalent to calculating `sqrt(sum((v - mean(v)).^2) / (length(v) - 1))`. Note: Julia does not ignore `NaN` values in the computation. For applications requiring the handling of missing data, the `DataArray` package is recommended.
